In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from sympy import *
from scipy.integrate import odeint

In the following cell I'm calculating $Y_{eq}$ for a W and Z bosons. \
I'm using boltzmann equation to calculate the number density $n$. 
$$n = \tfrac{g}{(2 \pi)^3} \int_{0}^{\inf} \tfrac{p^2}{exp(\sqrt{p^2+m^2}/T)-1} \,dx $$

$$Y_{eq} = n/T^3$$

In [7]:
#This function calculates the equilibrium abundance of a boson.
# T is temperature in Gev, M is mass of particle in Gev. 
#for w and Z bosons, M = 80 Gev. 
def Boltzmann_eq(p,T,m):
    integrand = p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1)
    return integrand

def Equilibrium_abundance(T,m,g):
    result = quad(Boltzmann_eq,0, 10000, args = (T,m))
    Value = ((9/(2*np.pi**2))*result[0])/T**3
    return Value

Equilibrium_abundance(80,80,9)



0.8106839724807485

This where I make a list of Yeq values.


In [13]:
#make Yeq list
Ts= np.logspace(3, 0, 100)
def Yeq_list(Tlist):
    Yeq = []
    for T in Tlist:
        Yeq.append(Equilibrium_abundance(T, 80,9))
    return np.array(Yeq)
Yeq_list(Ts)

/tmp/ipykernel_4604/1311843400.py:5: RuntimeWarning: overflow encountered in exp
  integrand = p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1)
/tmp/ipykernel_4604/1311843400.py:9: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result = quad(Boltzmann_eq,0, 10000, args = (T,m))


array([1.08819277e+00, 1.08863108e+00, 1.08851152e+00, 1.08799917e+00,
       1.08720299e+00, 1.08618769e+00, 1.08498569e+00, 1.08360736e+00,
       1.08204884e+00, 1.08029736e+00, 1.07833450e+00, 1.07613794e+00,
       1.07368228e+00, 1.07093927e+00, 1.06787787e+00, 1.06446408e+00,
       1.06066080e+00, 1.05642764e+00, 1.05172079e+00, 1.04649286e+00,
       1.04069271e+00, 1.03426544e+00, 1.02715229e+00, 1.01929070e+00,
       1.01061438e+00, 1.00105348e+00, 9.90534935e-01, 9.78982835e-01,
       9.66319049e-01, 9.52463972e-01, 9.37337500e-01, 9.20860242e-01,
       9.02954988e-01, 8.83548472e-01, 8.62573438e-01, 8.39971039e-01,
       8.15693555e-01, 7.89707445e-01, 7.61996678e-01, 7.32566325e-01,
       7.01446301e-01, 6.68695170e-01, 6.34403834e-01, 5.98698920e-01,
       5.61745621e-01, 5.23749677e-01, 4.84958187e-01, 4.45658891e-01,
       4.06177552e-01, 3.66873135e-01, 3.28130517e-01, 2.90350635e-01,
       2.53938120e-01, 2.19286759e-01, 1.86763387e-01, 1.56691134e-01,
      

This is where I calculate $\lambda$ eq 6.13 in Huterer

In [14]:
#This function calculates lambda in the expression for particle abundance
#where cross_section is particle cross_section, 
#m is mass, 
#gstar, is effective number of relativistic degress of freedom
#degrees of freedom is g 
def Lambda(cross_section,m,gstar,g):
    #planck mass in Gev 
    mpl = 1.220890* 10**19
    Hubble_constant = 1.66*g**(1/2)*(m**2/mpl)
    value = 2*(3.14)**2/45*gstar*((m**3*cross_section)/(Hubble_constant))
    return value
    
Lambda(10**5,80,86.25,9)

7.412642624149934e+26

This is the model I came up with by rewriting $dY/dx$ as $dlnY/lnx$

$$\tfrac{dLnY}{dLnx} = - \tfrac{\lambda}{x} \left[Y- \tfrac{Y_{eq}^2}{Y} \right]$$

In [15]:
def model(y,x):
    L = 7.412642624149934e+26
    dydx = -L/x * (y-Yeq_list(Ts)**2/y)
    return dydx


Log of the model I mentioned above.

In [25]:
def model2(y,x):
    L = 7.412642624149934e+26
    dydx = -L/np.log(x) * (np.log(y)-Yeq_list(Ts)[1]/np.log(y))
    return dydx


This is the equation given in the book that the question says I need to solve. eq 6.14 

In [30]:
def model3(y,x):
    L = 7.412642624149934e+26
    dydx = -L/x**2 * (y**2-Yeq_list(Ts)**2)
    return dydx

In [ ]:
plt.loglog(x,y)
plt.xlabel('time')
plt.ylabel('y(x)')
plt.show()

when I try to solve the ode of any of the 3 models, I get an error that relates to my model diverging. I do not know what to do about it. 

In [ ]:
# initial condition
y0 =Yeq_list(Ts)[0]*np.ones(100)

# time points
x = np.log(80)*np.ones(100)/np.logspace(2,0,100)
0
# solve ODE
y = odeint(model,y0,x)

print(y)

/tmp/ipykernel_4604/1311843400.py:5: RuntimeWarning: overflow encountered in exp
  integrand = p**2/(np.exp(np.sqrt(p**2+m**2)/T)-1)
/tmp/ipykernel_4604/1311843400.py:9: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result = quad(Boltzmann_eq,0, 10000, args = (T,m))


 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.1384768974573D-31
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.1384768974573D-31
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.2956655753361D-30
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.4382026634674D-01   r2 =  0.2956655753361D-30
 lsoda--  warning..internal t (=r1) 

In [ ]:
plt.loglog(x,Yeq_list(Ts))
x = np.log(80)*np.ones(100)/np.logspace(2,0,100)
plt.xlabel('time')
plt.ylabel('y(x)')
plt.show()